# Используем BERT впервые

Источник: [Jay Alamar](http://jalammar.github.io/a-visual-guide-to-using-bert-for-the-first-time/)


BERT --  это большой энкодер из трансформера, обученный предсказывать пропущенные слова в тексте.

<img src="https://www.researchgate.net/profile/Faiza_Khattak/publication/332543716/figure/fig3/AS:796161606684672@1566831127392/BERT-model-10-Taking-masked-input-and-outputting-the-masked-words.ppm" />


В этом семинаре мы будем использовать BERT (или DistilBERT), чтобы получить векторные представления для текста, а затем -- простую модель, чтобы решить задачу классификации. В качестве классификации мы будем решать задачу определения тональности.

### Models: Sentence Sentiment Classification

Так как мы занимаемся transfer learning, наша модель будет состоять из двух частей:

* DistilBERT processes the sentence and passes along some information it extracted from it on to the next model. DistilBERT is a smaller version of BERT developed and open sourced by the team at HuggingFace. It’s a lighter and faster version of BERT that roughly matches its performance.
* The next model, a basic Logistic Regression model from scikit learn will take in the result of DistilBERT’s processing, and classify the sentence as either positive or negative (1 or 0, respectively).

The data we pass between the two models is a vector of size 768. We can think of this of vector as an embedding for the sentence that we can use for classification.


<img src="https://jalammar.github.io/images/distilBERT/distilbert-bert-sentiment-classifier.png" />

## Dataset
The dataset we will use in this example is [SST2](https://nlp.stanford.edu/sentiment/index.html), which contains sentences from movie reviews, each labeled as either positive (has the value 1) or negative (has the value 0):


<table class="features-table">
  <tr>
    <th class="mdc-text-light-green-600">
    sentence
    </th>
    <th class="mdc-text-purple-600">
    label
    </th>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      a stirring , funny and finally transporting re imagining of beauty and the beast and 1930s horror films
    </td>
    <td class="mdc-bg-purple-50">
      1
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      apparently reassembled from the cutting room floor of any given daytime soap
    </td>
    <td class="mdc-bg-purple-50">
      0
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      they presume their audience won't sit still for a sociology lesson
    </td>
    <td class="mdc-bg-purple-50">
      0
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      this is a visually stunning rumination on love , memory , history and the war between art and commerce
    </td>
    <td class="mdc-bg-purple-50">
      1
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      jonathan parker 's bartleby should have been the be all end all of the modern office anomie films
    </td>
    <td class="mdc-bg-purple-50">
      1
    </td>
  </tr>
</table>

## Устанавливаем библиотеку transformers от huggingface

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.1MB 2.8MB/s 
     |████████████████████████████████| 3.0MB 16.0MB/s 
     |████████████████████████████████| 1.1MB 36.8MB/s 
     |████████████████████████████████| 890kB 37.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=b5179d06bca3c8563c32b5ea8fbc3428b82b0a2ed544f86e7fc683d105c0ab80
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

## Данные

In [ ]:
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)

In [ ]:
df.head()

,0,1
0,"a stirring , funny and finally transporting re...",1
1,apparently reassembled from the cutting room f...,0
2,they presume their audience wo n't sit still f...,0
3,this is a visually stunning rumination on love...,1
4,jonathan parker 's bartleby should have been t...,1


In [ ]:
df.shape

(6920, 2)

Возьмём первые 2,000.

In [ ]:
batch_1 = df[:2000]

Баланс классов:

In [ ]:
batch_1[1].value_counts()

1    1041
0     959
Name: 1, dtype: int64

## Loading the Pre-trained BERT model

In [ ]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Right now, the variable `model` holds a pretrained distilBERT model -- a version of BERT that is smaller, but much faster and requiring a lot less memory.


### Токенизация
Our first step is to tokenize the sentences -- break them up into word and subwords in the format BERT is comfortable with.


In [ ]:
tokenized = batch_1[0].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))

In [ ]:
tokenized[:5] # индексы токенов в классе токенизатора

0    [101, 1037, 18385, 1010, 6057, 1998, 2633, 182...
1    [101, 4593, 2128, 27241, 23931, 2013, 1996, 62...
2    [101, 2027, 3653, 23545, 2037, 4378, 24185, 10...
3    [101, 2023, 2003, 1037, 17453, 14726, 19379, 1...
4    [101, 5655, 6262, 1005, 1055, 12075, 2571, 376...
Name: 0, dtype: object

В начало каждого текста добавляется токен `[CLS]`. Его эмбеддинг будет служить эмбеддингом всего текста при классификации текстов.

In [ ]:
df.iloc[0][0]

'a stirring , funny and finally transporting re imagining of beauty and the beast and 1930s horror films'

In [ ]:
tokenizer.decode(tokenized[0])

'[CLS] a stirring, funny and finally transporting re imagining of beauty and the beast and 1930s horror films [SEP]'

In [ ]:
tokenizer.vocab['[CLS]']

101

Особенности токенизации: [WordPiece tokenization](https://medium.com/@makcedward/how-subword-helps-on-your-nlp-model-83dd1b836f46). Это эффективный способ бороться с OOV словами: если слова нет в словаре, разбей его на знакомые кусочки.

In [ ]:
tokenizer.wordpiece_tokenizer.tokenize('interesting work')

['interesting', 'work']

In [ ]:
tokenizer.wordpiece_tokenizer.tokenize('pythonista')

['python', '##ista']

In [ ]:
tokenizer.wordpiece_tokenizer.tokenize('cowork')

['cow', '##or', '##k']

<img src="https://jalammar.github.io/images/distilBERT/bert-distilbert-tokenization-2-token-ids.png" />

### Padding
Выравниваем предложения по длине с помощью нулевых токенов.

In [ ]:
# находим самое длинное предложение
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

# заполняем обучающие данные, где не хватает длины до максимума -- добавляем нули
padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [ ]:
np.array(padded).shape

(2000, 59)

In [ ]:
padded[0]

array([  101,  1037, 18385,  1010,  6057,  1998,  2633, 18276,  2128,
       16603,  1997,  5053,  1998,  1996,  6841,  1998,  5687,  5469,
        3152,   102,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0])

In [ ]:
tokenizer.ids_to_tokens[0]

'[PAD]'

### Masking

Теперь создаём отдельную переменную, чтобы сказать берту, что надо игнорировать паддинг при подсчёте attention.

In [ ]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(2000, 59)

In [ ]:
len(tokenized[0])

20

In [ ]:
attention_mask[0]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

## Используем BERT

`model()` прогоняет предложения через BERT.

In [ ]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

Let's slice only the part of the output that we need. That is the output corresponding the first token of each sentence. The way BERT does sentence classification, is that it adds a token called `[CLS]` (for classification) at the beginning of every sentence. The output corresponding to that token can be thought of as an embedding for the entire sentence.

<img src="https://jalammar.github.io/images/distilBERT/bert-output-tensor-selection.png" />

Берём оттуда только представления первого токена -- `[CLS]`. Это представление и будет нашими признаками.

In [ ]:
last_hidden_states[0][0]

tensor([[-0.2159, -0.1403,  0.0083,  ..., -0.1369,  0.5867,  0.2011],
        [-0.2471,  0.2468,  0.1008,  ..., -0.1631,  0.9349, -0.0715],
        [ 0.0558,  0.3573,  0.4140,  ..., -0.2430,  0.1770, -0.5080],
        ...,
        [-0.0165,  0.1179,  0.3512,  ..., -0.2401,  0.2722, -0.1750],
        [ 0.0961,  0.0667,  0.3147,  ..., -0.3277,  0.3556, -0.2135],
        [ 0.0454,  0.0519,  0.3168,  ..., -0.2880,  0.1844, -0.1042]])

In [ ]:
last_hidden_states[0].shape

torch.Size([2000, 59, 768])

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()

In [ ]:
features.shape

(2000, 768)

Метки:

In [ ]:
labels = batch_1[1]

## LogReg на признаках из BERT
Разделим данные на обучающую и тестовую выборки.

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

We now train the LogisticRegression model. If you've chosen to do the gridsearch, you can plug the value of C into the model declaration (e.g. `LogisticRegression(C=5.2)`).

In [ ]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

## Оцениваем результат
Accuracy на тесте:

In [ ]:
lr_clf.score(test_features, test_labels)

0.824

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(lr_clf.predict(test_features), test_labels))

              precision    recall  f1-score   support

           0       0.85      0.80      0.83       263
           1       0.79      0.85      0.82       237

    accuracy                           0.82       500
   macro avg       0.82      0.83      0.82       500
weighted avg       0.83      0.82      0.82       500



Сравним с DummyClassifier

In [ ]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.533 (+/- 0.03)


## Proper SST2 scores
For reference, the [highest accuracy score](http://nlpprogress.com/english/sentiment_analysis.html) for this dataset is currently **96.8**. DistilBERT can be trained to improve its score on this task – a process called **fine-tuning** which updates BERT’s weights to make it achieve a better performance in this sentence classification task (which we can call the downstream task). The fine-tuned DistilBERT turns out to achieve an accuracy score of **90.7**. The full size BERT model achieves **94.9**.


And that’s it! That’s a good first contact with BERT. The next step would be to head over to the documentation and try your hand at [fine-tuning](https://huggingface.co/transformers/examples.html#glue).

При желании можно вернуться и заменить distilBERT на BERT, чтобы посмотреть какие результаты будут у него.

### Как использовать BERT в качестве эмбеддингов слов

In [ ]:
s1 = 'i turn on the lights'
s2 = 'you turn into a beast'
s3 = 'turn on the air condition'

Токенизируем

In [ ]:
s1_tok = tokenizer.encode(s1)
s2_tok = tokenizer.encode(s2)
s3_tok = tokenizer.encode(s3)

Получаем эмбеддинги

In [ ]:
with torch.no_grad():
    last_hidden_states1 = model(torch.tensor([s1_tok]))
    last_hidden_states2 = model(torch.tensor([s2_tok]))
    last_hidden_states3 = model(torch.tensor([s3_tok]))

Находим индексы интересующих нас токенов

In [ ]:
word_index = tokenizer.vocab['turn']
ind1, ind2, ind3 = s1_tok.index(word_index), s2_tok.index(word_index), s3_tok.index(word_index)

In [ ]:
ind1, ind2, ind3

(2, 2, 1)

Вот эмбеддинги, которые нам нужны

In [ ]:
word1_emb = last_hidden_states1[0][:,ind1,:][0]
word2_emb = last_hidden_states2[0][:,ind2,:][0]
word3_emb = last_hidden_states3[0][:,ind3,:][0]

Теперь посчитаем расстояние:

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# bank из первого предложения дальше всех от остальных
cosine_similarity([
    word1_emb.numpy(),
    word2_emb.numpy(),
    word3_emb.numpy()
])

array([[1.        , 0.59958804, 0.6553918 ],
       [0.59958804, 0.99999994, 0.51167804],
       [0.6553918 , 0.51167804, 0.9999999 ]], dtype=float32)

## Русский -- DeepPavlov

In [ ]:
tokenizer = ppb.AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")

In [ ]:
model = ppb.AutoModel.from_pretrained("DeepPavlov/rubert-base-cased")

In [ ]:
tokenizer.wordpiece_tokenizer.tokenize('Русский текст')

['Русский', 'текст']

In [ ]:
tokenizer.wordpiece_tokenizer.tokenize('печенюшка')

['печен', '##юшка']

#### Задание: классификация ленты

In [ ]:
! wget -O lenta-ru-train.csv https://www.dropbox.com/s/kdupcw1llbdbqwl/lenta-ru-train.csv?dl=0

--2020-10-12 22:55:14--  https://www.dropbox.com/s/kdupcw1llbdbqwl/lenta-ru-train.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.66.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.66.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/kdupcw1llbdbqwl/lenta-ru-train.csv [following]
--2020-10-12 22:55:14--  https://www.dropbox.com/s/raw/kdupcw1llbdbqwl/lenta-ru-train.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc876ff7315d53620cb0404a3223.dl.dropboxusercontent.com/cd/0/inline/BBJHIdeUzoLLbdWFvq2uNL109KjSUPLeuJctKxe-EY3vgpgAw4-60uqTbhxWyFxmb_zwkW7MXzghy3R3-TtzLKLFnG0g5ydk2MDa2_jjWjKtomJQjUhPtWkQs9mhb0al75U/file# [following]
--2020-10-12 22:55:15--  https://uc876ff7315d53620cb0404a3223.dl.dropboxusercontent.com/cd/0/inline/BBJHIdeUzoLLbdWFvq2uNL109KjSUPLeuJctKxe-EY3vgpgAw4-60uqTbhxWyFxmb_zwkW7MXzghy3R3-TtzLKLF

In [ ]:
df_lenta = pd.read_csv('lenta-ru-train.csv')

In [ ]:
df_lenta.head()

,title,text,topic,topic_label
0,"Инфляция в январе 2006 года составит 2,6 процента","Глава Росстата Владимир Соколин заявил, что в ...",Экономика,0
1,Никита Михалков учредил День российского кино,У российских кинематографистов появится новый...,Культура,3
2,Марко Матерацци вернется в строй к матчу с ЦСКА,"Медицинский штаб миланского ""Интера"" обнародов...",Спорт,1
3,"Определены лауреаты премии ""Книга года""","Премии ""Книга года"" в 13 номинациях вручены на...",Культура,3
4,Гол Роналду со штрафного спас португальцев от ...,Сборная Португалии сыграла вничью с командой И...,Спорт,1


#### Сначала надо написать извлечение эмбеддингов из BERT для графы title

#### А потом использовать эти эмбеддинги для классификации

#### Альтернативное задание: классификация IMDB 



In [ ]:
!wget https://raw.githubusercontent.com/devkosal/fastai_roberta/master/fastai_roberta_imdb/imdb_dataset.csv